In [ ]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


In [ ]:
!pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

In [ ]:
import os
from typing import Optional

import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from adaptive_lora.callbacks import AdaptiveLoRACallback

In [ ]:
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0" 
output_dir = "./tinyllama-qnli-lora"
seed = 42
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

In [ ]:
batch_size = 16
eval_batch_size = 32
num_train_epochs = 3
max_length = 128
learning_rate = 2e-5
weight_decay = 0.01
rank=8
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=rank,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ],)

In [ ]:

def clean_text(s: Optional[str]) -> str:
    if s is None:
        return ""
    return " ".join(str(s).strip().split())

def build_one_shot_demo(example: dict) -> str:
    q = clean_text(example["question"])
    c = clean_text(example["context"])
    lbl = example.get("label_text", "Yes")
    return f"Example:\nQuestion: {q}\nContext: {c}\nAnswer (Yes/No): {lbl}\n\n"

In [ ]:
ONE_SHOT_EXAMPLE = {
    "question": "Who wrote Hamlet?",
    "context": "Hamlet was written by William Shakespeare and first performed in the early 17th century.",
    "label_text": "Yes",  # "Yes" => the context contains the answer
}


dataset = load_dataset("glue", "qnli")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

if tokenizer.pad_token is None:
    # safe default: use eos_token as pad
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:

# Load tokenizer


def preprocess_function(examples, *, max_length: int = max_length, add_demo: bool = False):
    """
    Build instruction-like prompts and tokenize them.
    - add_demo: if True, prepends a 1-shot example to each prompt (can help decoder-only models)
    """
    questions = [clean_text(q) for q in examples["question"]]
    contexts = [clean_text(s) for s in examples["sentence"]]

    demo_str = build_one_shot_demo(ONE_SHOT_EXAMPLE) if add_demo else ""

    prompts = []
    for q, c in zip(questions, contexts):
        # Instruction-style prompt ending with a short, constrained label target
        prompt = (
            "You are a helpful assistant.\n"
            f"{demo_str}"
            f"Question: {q}\n"
            f"Context: {c}\n"
            "Answer (Yes/No):"
        )
        prompts.append(prompt)

    tokenized = tokenizer(
        prompts,
        truncation=True,
        max_length=max_length,
        padding=False, 
    )

    if "label" in examples:
        tokenized["labels"] = examples["label"]
    elif "labels" in examples:
        tokenized["labels"] = examples["labels"]

    return tokenized

print("Tokenizing dataset...")
tokenized = dataset.map(
    lambda ex: preprocess_function(ex, max_length=max_length, add_demo=False),
    batched=True,
    remove_columns=["question", "sentence", "idx"],
)

if "label" in tokenized["train"].column_names and "labels" not in tokenized["train"].column_names:
    tokenized = tokenized.rename_column("label", "labels")

tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest", pad_to_multiple_of=8)
train_dataset = tokenized["train"].select(range(10000))
eval_dataset = tokenized["validation"].select(range(1500))

In [ ]:
print("Loading model...")
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

In [ ]:

# Resize token embeddings if tokenizer changed
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

# -------------------------
# Apply LoRA (PEFT)
# -------------------------
print("Applying LoRA (PEFT)...")
model = get_peft_model(model, peft_config)

In [ ]:
val_dataloader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
    collate_fn=data_collator,
    pin_memory=torch.cuda.is_available(),
    num_workers=2,
)

In [ ]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    acc = accuracy.compute(predictions=preds, references=labels)
    return {"accuracy": acc["accuracy"]}

# -------------------------
# TrainingArguments + Trainer
# -------------------------
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=eval_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    num_train_epochs=num_train_epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    seed=seed,
    fp16=False,
    bf16=True,
    push_to_hub=False,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Starting training...")
trainer.train()
print("Saving model and adapters...")
trainer.save_model(output_dir)
print("Saved model to", output_dir)

In [ ]:
results = trainer.evaluate()
print(results)